# Пример использования DataFrameProcessorFramework

Этот пример демонстрирует основные возможности фреймворка для обработки DataFrame в Google Colab.

In [ ]:
# Импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import sys
from datetime import datetime, timedelta

## 1. Подключение фреймворка

Сначала нужно добавить путь к фреймворку или установить его.

In [ ]:
# Вариант 1: Если фреймворк уже скопирован в директорию Colab
sys.path.append('/content/Data_prep')  # Измените путь при необходимости

# Вариант 2: Установка из репозитория GitHub (раскомментируйте при необходимости)
# !git clone https://github.com/username/Data_prep.git
# sys.path.append('/content/Data_prep')

## 2. Создание тестового DataFrame

Для демонстрации создадим тестовый DataFrame с различными типами данных и пропущенными значениями.

In [ ]:
# Создаем тестовый набор данных
def create_test_dataframe():
    # Создаем даты
    base_date = datetime.now()
    dates = [base_date + timedelta(days=i) for i in range(100)]
    
    # Создаем числовые данные с пропусками
    np.random.seed(42)
    values = np.random.normal(100, 15, 100)
    values[np.random.choice(100, 15, replace=False)] = np.nan  # Добавляем пропуски
    
    # Создаем категориальные данные с пропусками
    categories = np.random.choice(['A', 'B', 'C', 'D', np.nan], 100)
    
    # Создаем DataFrame
    df = pd.DataFrame({
        'timestamp': dates,
        'value': values,
        'category': categories,
        'integer': np.random.randint(1, 100, 100),
        'decimal': np.random.random(100) * 100
    })
    
    # Добавляем дубликаты
    duplicate_rows = df.sample(10).copy()
    df = pd.concat([df, duplicate_rows]).reset_index(drop=True)
    
    # Преобразуем timestamp в строку, чтобы имитировать сырые данные
    df['timestamp'] = df['timestamp'].astype(str)
    
    return df

# Создаем тестовый DataFrame
raw_dataframe = create_test_dataframe()

# Проверяем исходные данные
print("Размер данных:", raw_dataframe.shape)
print("\nИнформация о данных:")
raw_dataframe.info()
print("\nПервые 5 строк:")
display(raw_dataframe.head())

# Проверяем наличие пропусков
print("\nКоличество пропущенных значений:")
display(raw_dataframe.isna().sum())

## 3. Запуск UI для обработки данных

Теперь запустим интерактивный интерфейс для обработки нашего DataFrame.

In [ ]:
# Импортируем функцию запуска UI
from dataframe_processor_framework import launch_processor_ui

# Запускаем UI
processor_ui_instance = launch_processor_ui(raw_dataframe)

## 4. Получение обработанного DataFrame

После работы с UI и нажатия кнопки "Обработать и сохранить", получаем обработанный DataFrame.

In [ ]:
# Получаем обработанный DataFrame
processed_df = processor_ui_instance.get_final_dataframe()

# Проверяем результат
if processed_df is not None:
    print("Обработанный DataFrame готов к использованию!")
    print("\nРазмер данных после обработки:", processed_df.shape)
    print("\nИнформация об обработанных данных:")
    processed_df.info()
    print("\nПервые 5 строк обработанного DataFrame:")
    display(processed_df.head())
    
    # Проверяем наличие пропусков после обработки
    print("\nКоличество пропущенных значений после обработки:")
    display(processed_df.isna().sum())
else:
    print("Обработка не была завершена или DataFrame не доступен.")

## 5. Прямое использование компонентов фреймворка

При необходимости можно использовать компоненты фреймворка напрямую без UI.

In [ ]:
# Пример прямого использования DataFrameProcessor
from dataframe_processor_framework import DataFrameProcessor

# Создаем экземпляр процессора
processor = DataFrameProcessor(raw_dataframe.copy())

# Применяем последовательно операции обработки
processor.set_datetime_index(time_col='timestamp')
processor.handle_missing_values(strategy='ffill')
processor.remove_duplicate_rows(keep='first')
processor.convert_data_types({'decimal': 'float', 'integer': 'int'})

# Получаем результат
manually_processed_df = processor.get_processed_df()

# Проверяем результат
print("DataFrame после ручной обработки:")
display(manually_processed_df.head())

# Получаем лог операций
print("\nЛог операций:")
for msg in processor.get_log_messages():
    print("- " + msg)